### We want to convert text --> numerical values
#### 1. We need a Vocabulary mapping each word to a index
#### 2. We need to setup a Pytorch dataset to load the data
#### 3. Setup padding of every batch ( all examples should be of same seq_len and setup dataloader)

In [1]:
# Import
import os
import pandas as pd 
import spacy
import torch
from torch.nn.utils.rnn import pad_sequence # pad batch
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as transforms

In [2]:
spacy_eng = spacy.load('en_core_web_sm')

In [9]:
text = 'I love you'
tok = spacy_eng.tokenizer(text)
[t.text.lower() for t in tok]

['i', 'love', 'you']

In [ ]:
! python -m spacy download en

In [3]:
class Vocabulary:
    def __init__(self, freq_threshold):
        ''' "<PAD>": padding_value, value for padded elements in pad_sequence
            "<SOS>": Start of sentence
            "<EOS>": End of sentence
            "<UNK>": Unknow token  '''
        self.itos = {0:"<PAD>", 1:"<SOS>", 2:"<EOS>", 3:"<UNK>"}
        self.stoi = {"<PAD>":0, "<SOS>":1, "<EOS>":2, "<UNK>":3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)
    
    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]
    
    # "I love peanuts" --> ["i", "love", "peanuts"]
    
    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1
                
                else:
                    frequencies[word] += 1
                
                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word 
                    idx += 1
    
    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]


In [15]:
captions = pd.read_csv(r"E:\Learn_something\Pytorch_Getting_start\dataset\flickr8k\captions.txt")
print(captions['caption'].tolist()[:5])


['A child in a pink dress is climbing up a set of stairs in an entry way .', 'A girl going into a wooden building .', 'A little girl climbing into a wooden playhouse .', 'A little girl climbing the stairs to her playhouse .', 'A little girl in a pink dress going into a wooden cabin .']


In [4]:
class FlickrDataset(Dataset):
    def __init__(self, root_dir, captions_file,
                 transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(captions_file)
        self.transform = transform

        # Get img, caption columns
        self.imgs = self.df["image"]
        self.captions = self.df["caption"]

        # Initialize vocabulary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)
        
        numericalized_caption = [self.vocab.stoi["<SOS>"]] # stoi: string to index; SOS: start of sentence
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"]) # EOS: end of sentence

        return img, torch.tensor(numericalized_caption)


In [5]:
class MyCollate:
    '''pad everything to that specific length'''
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)

        return imgs, targets

In [6]:
def get_loader(
    root_folder,
    annotation_file,
    transform,
    batch_size=32,
    num_workers=8,
    shuffle=True,
    pin_memory=True,
):
    dataset = FlickrDataset(root_folder, annotation_file, transform=transform)

    pad_idx = dataset.vocab.stoi["<PAD>"]

    loader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        shuffle=shuffle,
        pin_memory=pin_memory,
        collate_fn=MyCollate(pad_idx=pad_idx),
    )

    return loader, dataset

In [7]:
# jupyter 运行不出来，.py文件可以正常运行, 目前没找到问题
if __name__ == "__main__":
    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ]
    )

    loader, dataset = get_loader(
            r"E:\Learn_something\Pytorch_Getting_start\dataset\flickr8k\images/", 
            r"E:\Learn_something\Pytorch_Getting_start\dataset\flickr8k\captions.txt", 
            transform=transform)

    # for idx, (imgs, captions) in enumerate(loader):
    #     print(imgs.shape)
    #     print(captions.shape)
    imgs, captions = next(iter(loader))
    print(imgs.shape)
    print(captions.shape)